In [6]:
### 2024 SEP 6 ### script4_seq ## matching

In [7]:
##
# conda install -c conda-forge r-optmatch
##

In [1]:
install.packages("s3")
install.packages("doParallel")

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done



In [2]:
library("terra")
library("dplyr")
library("sf")
#install.packages("s3")
library("s3")

library("sp")
library("foreach")
library("stringr")
library("aws.s3")
#conda install -c conda-forge r-optmatch #r-ggmap r-hrbrthemes r-Hmisc
library("optmatch")
#install.packages("doParallel")
library("doParallel")


terra 1.7.29


Attaching package: ‘dplyr’


The following objects are masked from ‘package:terra’:

    intersect, union


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Linking to GEOS 3.11.2, GDAL 3.7.0, PROJ 9.2.0; sf_use_s2() is TRUE

Loading required package: iterators

Loading required package: parallel



In [3]:
#To test, we define the variables manually. For final version, run the commented out section below
#iso3 <-"Bpt"
#iso3 <-"Bce"
#iso3 <-"Bca"
#iso3 <- "Bma"
#iso3 <- "BaE"
#iso3 <- "BaW"
iso3 <- "GHA"
gediwk <- 24
mproc <- 1

In [4]:
#-------------------------------------------------------------------------------
#args = commandArgs(trailingOnly=TRUE)
#if (length(args)==0) {
#  stop("At least one argument must be supplied (input file).n", call.=FALSE)
#} else if (length(args)>=1) {
#  
#  iso3 <- args[1]  #country to process
#  gediwk <- args[2]   #the # of weeks GEDI data to use
#  mproc <- as.integer(args[3])  #the number of cores to use for matching 
#}
#-------------------------------------------------------------------------------

In [5]:
cat("Step 0: Loading global variables for", iso3,"with wk", gediwk, "data \n")

#f.path <- "/projects/my-public-bucket/GEDI_global_PA_v2/"
f.path <- "s3://maap-ops-workspace/shared/leitoldv/GEDI_global_PA_v2/"

matching_tifs <- c("wwf_biomes","wwf_ecoreg","lc2000","d2roads", "dcities","dem",
                   "pop_cnt_2000","pop_den_2000","slope", "tt2cities_2000", "wc_prec_1990-1999",
                   "wc_tmax_1990-1999","wc_tavg_1990-1999","wc_tmin_1990-1999" )

ecoreg_key <- read.csv(s3_get(paste(f.path,"wwf_ecoregions_key.csv",sep="")))
#unlink(s3_get(paste(f.path,"wwf_ecoregions_key.csv",sep="")))

allPAs <- readRDS(s3_get(paste(f.path,"WDPA_shapefiles/WDPA_polygons/",iso3,"_PA_poly.rds",sep="")))

MCD12Q1 <- rast(s3_get(paste(f.path,"GEDI_ANCI_PFT_r1000m_EASE2.0_UMD_v1_projection_defined_6933.tif",sep="")))
crs(MCD12Q1)  <- "epsg:6933"

world_region <- rast(s3_get(paste(f.path,"GEDI_ANCI_CONTINENT_r1000m_EASE2.0_UMD_v1_revised_projection_defined_6933.tif",sep="")))
crs(world_region)  <- "epsg:6933"

s3_get_files(c(paste(f.path,"WDPA_countries/shp/",iso3,".shp",sep=""),
              paste(f.path,"WDPA_countries/shp/",iso3,".shx",sep=""),
              paste(f.path,"WDPA_countries/shp/",iso3,".prj",sep=""),
              paste(f.path,"WDPA_countries/shp/",iso3,".dbf",sep="")),confirm = FALSE)
adm <- st_read(s3_get(paste(f.path,"WDPA_countries/shp/",iso3,".shp",sep="")))

#s3_path <- paste("/vsis3/maap-ops-workspace/shared/leitoldv/GEDI_global_PA_v2/WDPA_countries/shp/",iso3,".shp",sep="") #Redo this for the gpkg
#adm <- st_read(s3_path)
adm_prj <- project(vect(adm), "epsg:6933")

load(s3_get(paste(f.path,"rf_noclimate.RData",sep="")))
#source(s3_get(paste(f.path,"matching_func.R",sep="")))
source(s3_get(paste(f.path,"matching_func_2024.R",sep="")))


Step 0: Loading global variables for GHA with wk 24 data 


ℹ s3://maap-ops-workspace/shared/leitoldv/GEDI_global_PA_v2/wwf_ecoregions_key.csv already exists at /projects/.local/share/R/s3/maap-ops-workspace/shared/leitoldv/GEDI_global_PA_v2/wwf_ecoregions_key.csv

ℹ s3://maap-ops-workspace/shared/leitoldv/GEDI_global_PA_v2/GEDI_ANCI_PFT_r1000m_EASE2.0_UMD_v1_projection_defined_6933.tif already exists at /projects/.local/share/R/s3/maap-ops-workspace/shared/leitoldv/GEDI_global_PA_v2/GEDI_ANCI_PFT_r1000m_EASE2.0_UMD_v1_projection_defined_6933.tif

ℹ s3://maap-ops-workspace/shared/leitoldv/GEDI_global_PA_v2/GEDI_ANCI_CONTINENT_r1000m_EASE2.0_UMD_v1_revised_projection_defined_6933.tif already exists at /projects/.local/share/R/s3/maap-ops-workspace/shared/leitoldv/GEDI_global_PA_v2/GEDI_ANCI_CONTINENT_r1000m_EASE2.0_UMD_v1_revised_projection_defined_6933.tif

ℹ 4 files totaling 22.22 kB will be downloaded to /projects/.local/share/R/s3

→ Downloading 4 files.

→ Got 0 files, downloading 4

→ Got 1 file, downloading 3

→ Got 2 files, downloading 2

Reading layer `GHA' from data source 
  `/projects/.local/share/R/s3/maap-ops-workspace/shared/leitoldv/GEDI_global_PA_v2/WDPA_countries/shp/GHA.shp' 
  using driver `ESRI Shapefile'
Simple feature collection with 1 feature and 94 fields
Geometry type: POLYGON
Dimension:     XY
Bounding box:  xmin: -3.262509 ymin: 4.737128 xmax: 1.187968 ymax: 11.16294
Geodetic CRS:  WGS 84


ℹ s3://maap-ops-workspace/shared/leitoldv/GEDI_global_PA_v2/rf_noclimate.RData already exists at /projects/.local/share/R/s3/maap-ops-workspace/shared/leitoldv/GEDI_global_PA_v2/rf_noclimate.RData

ℹ s3://maap-ops-workspace/shared/leitoldv/GEDI_global_PA_v2/matching_func_2024.R already exists at /projects/.local/share/R/s3/maap-ops-workspace/shared/leitoldv/GEDI_global_PA_v2/matching_func_2024.R



In [6]:
#STEP4. Set up spatial points data frames (control + each PA) for point matching
# if (file.exists(paste(f.path,"WDPA_matching_results/",iso3,"_wk",gediwk,"/",iso3,"_matching_output_wk",gediwk,".RDS", sep=""))){
d_control_local <- readRDS(s3_get(paste(f.path,"WDPA_matching_points/",iso3,"/",iso3,"_prepped_control_wk",gediwk,".RDS",sep="")))
d_control_local <- d_control_local[complete.cases(d_control_local), ]  #filter away non-complete cases w/ NA in control set


In [7]:
print(d_control_local[1,])

                                         wwfbiom               wwfecoreg
1 Tropical & Subtropical Moist Broadleaf Forests Eastern Guinean forests
      land_cover    d2road    d2city elevation   popcnt   popden     slope
1 l3_agriculture 0.1118034 0.8408408       236 55.48445 64.96677 0.9363867
   tt2city     prec max_temp mean_temp min_temp       lon      lat status UID
1 199.5314 3850.611  34.3479  26.45991  20.6888 -3.106846 7.050933  FALSE   1


In [8]:
f.path <- "s3://maap-ops-workspace/shared/leitoldv/GEDI_global_PA_v2/"
testPAs_fileindex <- read.csv(s3_get(paste(f.path,"WDPA_matching_points/",iso3,"/",iso3,"_testPAs_fileindex.csv",sep="")))
d_PAs <- testPAs_fileindex[!is.na(testPAs_fileindex[,"filename"]),]$filename
d_PAs[1]
length(d_PAs)

ERROR: Error in s3_check_for_file_s3(s3_uri, region, public, data_dir = data_dir): file not found, check the URI; do you need AWS credentials for this file?


In [17]:
#f.path <- "/projects/my-public-bucket/GEDI_global_PA_v2/"
#f.path <- "s3://maap-ops-workspace/shared/leitoldv/GEDI_global_PA_v2/"
#dir.create(file.path(paste(f.path,"WDPA_matching_results/",iso3,"_wk",gediwk,"/",sep="")))

In [10]:
f.path <- "/projects/my-public-bucket/GEDI_global_PA_v2/"

if(!dir.exists(paste(f.path,"WDPA_matching_results/",iso3,"_wk",gediwk,"/",sep=""))){
  # cat("Matching result dir does not EXISTS\n")
  dir.create(file.path(paste(f.path,"WDPA_matching_results/",iso3,"_wk",gediwk,"/",sep="")))
#  d_PAs <- list.files(paste(f.path,"WDPA_matching_points/",iso3,"/",iso3,"_testPAs/", sep=""), pattern=paste("wk",gediwk,sep=""), full.names=FALSE)
  d_PAs <- testPAs_fileindex[!is.na(testPAs_fileindex[,"filename"]),]$filename

} else if (dir.exists(paste(f.path,"WDPA_matching_results/",iso3,"_wk",gediwk,"/",sep=""))){   #if matching result folder exists, check for any PAs w/o matched results
  pattern1 = c(paste("wk",gediwk,sep=""),"RDS")
  matched_PAid <- list.files(paste(f.path,"WDPA_matching_results/",iso3,"_wk",gediwk,"/",sep=""), full.names = FALSE, pattern=paste0(pattern1, collapse="|"))%>%
    readr::parse_number() %>% unique()
  #d_PAs <- list.files(paste(f.path,"WDPA_matching_points/",iso3,"/",iso3,"_testPAs/", sep=""), pattern=paste("wk",gediwk,sep=""), full.names=FALSE)
  d_PAs <- testPAs_fileindex[!is.na(testPAs_fileindex[,"filename"]),]$filename
  d_PA_id <- d_PAs %>% readr::parse_number()
  runPA_id <- d_PA_id[!(d_PA_id %in% matched_PAid)]
  if (length(runPA_id)>0){
    Pattern2 <-  paste(runPA_id, collapse="|")
    runPA <-  d_PAs[grepl(Pattern2,d_PAs)]
    d_PAs <- runPA
  } else {
    d_PAs <- NULL
  }
}
print(length(d_PAs))

ERROR: Error in eval(expr, envir, enclos): object 'testPAs_fileindex' not found


In [19]:
d_PAs

[1] "BaE_prepped_pa_478536_wk24.RDS"

In [14]:
#d_PAs <- d_PAs[length(d_PAs):1]


In [20]:
#this_pa <- d_PAs[1]
this_pa <- 'BaE_prepped_pa_352212_wk24.RDS'
pa <- this_pa
  #id_pa <- pa %>%str_split("_") %>% unlist %>% .[[3]]
  id_pa <- this_pa%>%readr::parse_number() %>% unique() #%>%str_split("_") %>% unlist %>% .[4] #With new files, check where PA ID is in string
  cat(id_pa, "in",iso3,"\n")
  cat("No.", match(pa,d_PAs),"of total",length(d_PAs),"PAs in ", iso3, "\n" )
  d_pa <- readRDS(paste(f.path,"WDPA_matching_points/",iso3,"/",iso3,"_testPAs/",pa, sep=""))
  cat(iso3, "pa no.",id_pa, "has",nrow(d_pa)," of treatment \n")
  d_filtered_prop <- tryCatch(propensity_filter(d_pa, d_control_local), error=function(e) return(NA))  #return a df of control and treatment after complete cases and propensity filters are applied
  cat("Propensity score filtered DF dimension is",dim(d_filtered_prop),"\n")
  d_wocat_all <- tryCatch(filter(d_filtered_prop, status),error=function(e) return(NA))
  d_control_all <- tryCatch(filter(d_filtered_prop, !status),error=function(e) return(NA))
  

352212 in BaE 
No. NA of total 1 PAs in  BaE 
BaE pa no. 352212 has 3  of treatment 
Propensity score filtered DF dimension is 0 19 


In [21]:
n_control <- dim(d_control_all)[1]
  # ids_all <- d_control_all$UID   #seq(1,n_control)
  ids_all0 <- tryCatch(d_control_all$UID, error=function(e) return(NA))
  ids_all <- d_control_all$UID
  set.seed(125)
  cat("Using number of cores:",getDoParWorkers(),"\n")
  N <- ceiling(nrow(d_wocat_all)/300)
  l <- tryCatch(split(d_wocat_all, sample(1:N, nrow(d_wocat_all), replace=TRUE)),error=function(e) return(NULL))
length(l)

Using number of cores: 1 


[1] 0

In [22]:
pa_match <- data.frame()
    

In [23]:
for (pa_c in 1:length(l)){
      ids_all <- d_control_all$UID
      cat("chunk",pa_c,"of PA", id_pa,"\n")
      
      d_wocat_chunk <- l[[pa_c]]
      # #sample the control dataset to the size of the sample dataset, keep unsampled ids to iterate until full number of matches found
      n_treatment <- dim(d_wocat_chunk)[1]
      
      t <- ifelse(floor(n_control/n_treatment)<=7, ifelse(floor(n_control/n_treatment)<1, 1,floor(n_control/n_treatment)),7)   #floor(n_control/n_treatment))
      n_sample <- round(n_treatment*t)    #now the n_control is 1.4 times the number of n_treatment, 7 will set the if ststament below to flase
      m_all2_out <- data.frame()
      Bscore <- data.frame()
      n_matches <- 0
      tryCatch(
        while(n_matches < n_treatment){
          n_ids <- length(ids_all)
          cat("n ids",n_ids,"\n")
          if(n_ids > n_sample){
            set.seed(125)
            sample_ids_bar <- sample(ids_all, n_sample)
            sample_ids <- sample(ids_all0, n_sample)
            d_control_sample <- d_control_all[d_control_all$UID %in% sample_ids,]
            ids_all <-setdiff(ids_all, sample_ids_bar)    #ids_all[-sample_ids]
            cat("protected uid", head(d_wocat_chunk$UID),"\n")
            # All approaches
            new_d <- tryCatch(rbind(d_wocat_chunk,d_control_sample),error=function(e) return(NULL))
            # new_d <- tryCatch(rbind(d_wocat_chunk,d_control_all),error=function(e) return(NULL))
            #create a smaller distance matrix
            m_all <- tryCatch(match_wocat(new_d, pid=id_pa),error=function(e) return(NULL))
            # m_all <- match_wocat(new_d)
            m_all2 <- tryCatch(m_all[1,],error=function(e) return(NULL))
            # m_all2 <- m_all[1,]
            n_matches_temp <- tryCatch(nrow(m_all2$df),error=function(e) return(NULL))
            # n_matches_temp <- nrow(m_all2$df)
            if(!is.null(n_matches_temp)){
              # n_matches <- n_matches + nrow(m_all2$df)
              m_all2$df$pa_id <- rep(id_pa,n_matches_temp)
              m_all2_out <- rbind(m_all2_out, m_all2$df)
              matched_protected <- m_all2$df %>% dplyr::filter(status==TRUE)
              matched_control <- m_all2$df %>% dplyr::filter(status==FALSE)
              cat("matched_protected", nrow(matched_protected),"\n")
              n_matches <- n_matches + nrow(matched_protected)
              d_wocat_chunk <- d_wocat_chunk[-(match(matched_protected$UID,d_wocat_chunk$UID)),]
              # d_control_all <- d_control_all[-(match(matched_control$UID,d_control$UID)),]
            } 
            # ids_all <-setdiff(ids_all, sample_ids)
            ids_all0 <-setdiff(ids_all0, matched_control$UID)
            # else {
            #   n_treatment <- 0  #if not macthes are found in this sampling
            # }
          } else {n_treatment <- n_matches}
        }, error=function(e) return(NULL))
      # ids_all0 <-setdiff(ids_all0, matched_control$UID)
      match_score <- m_all2_out
      cat(table(match_score$status),"\n")
      pa_match <- rbind(pa_match,match_score)
    }

chunk 1 of PA 352212 


ERROR: Error in l[[pa_c]]: subscript out of bounds


In [ ]:
#Initialize an empty list to store filenames
#output_filenames <- cbind(d_PAs, rep(NA, length(d_PAs)))
#colnames(output_filenames) <- c("prepped_PA", "matched_PA")
#matched_PA <- c()

registerDoParallel(mproc)
# cat("Parallel processing",getDoParWorkers(),"PAs \n")
startTime <- Sys.time()
#d_PAs <- list.files(paste(f.path,"WDPA_matching_points/",iso3,"/",iso3,"_testPAs/", sep=""), pattern=paste("wk",gediwk,sep=""), full.names=FALSE)

foreach(this_pa=d_PAs,.combine = foreach_rbind, .packages=c('sp','magrittr', 'dplyr','tidyr','optmatch','doParallel')) %dopar% {
  pa <- this_pa
  #id_pa <- pa %>%str_split("_") %>% unlist %>% .[[3]]
  id_pa <- this_pa%>%readr::parse_number() %>% unique() #%>%str_split("_") %>% unlist %>% .[4] #With new files, check where PA ID is in string
  cat(id_pa, "in",iso3,"\n")
  cat("No.", match(pa,d_PAs),"of total",length(d_PAs),"PAs in ", iso3, "\n" )
  d_pa <- readRDS(paste(f.path,"WDPA_matching_points/",iso3,"/",iso3,"_testPAs/",pa, sep=""))
  cat(iso3, "pa no.",id_pa, "has",nrow(d_pa)," of treatment \n")
  d_filtered_prop <- tryCatch(propensity_filter(d_pa, d_control_local), error=function(e) return(NA))  #return a df of control and treatment after complete cases and propensity filters are applied
  cat("Propensity score filtered DF dimension is",dim(d_filtered_prop),"\n")
  d_wocat_all <- tryCatch(filter(d_filtered_prop, status),error=function(e) return(NA))
  d_control_all <- tryCatch(filter(d_filtered_prop, !status),error=function(e) return(NA))
  
  n_control <- dim(d_control_all)[1]
  # ids_all <- d_control_all$UID   #seq(1,n_control)
  ids_all0 <- tryCatch(d_control_all$UID, error=function(e) return(NA))
  ids_all <- d_control_all$UID
  set.seed(125)
  cat("Using number of cores:",getDoParWorkers(),"\n")
  N <- ceiling(nrow(d_wocat_all)/300)
  l <- tryCatch(split(d_wocat_all, sample(1:N, nrow(d_wocat_all), replace=TRUE)),error=function(e) return(NULL))
  # l <- tryCatch(split(d_wocat_all, (as.numeric(rownames(d_wocat_all))-1) %/% 300),error=function(e) return(0))
  
#  if (length(l)<50 && length(l)>0 ){
  if (length(l)>0 ){
    pa_match <- data.frame()
    for (pa_c in 1:length(l)){
      ids_all <- d_control_all$UID
      cat("chunk",pa_c,"of PA", id_pa,"\n")
      
      d_wocat_chunk <- l[[pa_c]]
      # #sample the control dataset to the size of the sample dataset, keep unsampled ids to iterate until full number of matches found
      n_treatment <- dim(d_wocat_chunk)[1]
      
      t <- ifelse(floor(n_control/n_treatment)<=7, ifelse(floor(n_control/n_treatment)<1, 1,floor(n_control/n_treatment)),7)   #floor(n_control/n_treatment))
      n_sample <- round(n_treatment*t)    #now the n_control is 1.4 times the number of n_treatment, 7 will set the if ststament below to flase
      m_all2_out <- data.frame()
      Bscore <- data.frame()
      n_matches <- 0
      tryCatch(
        while(n_matches < n_treatment){
          n_ids <- length(ids_all)
          cat("n ids",n_ids,"\n")
          if(n_ids > n_sample){
            set.seed(125)
            sample_ids_bar <- sample(ids_all, n_sample)
            sample_ids <- sample(ids_all0, n_sample)
            d_control_sample <- d_control_all[d_control_all$UID %in% sample_ids,]
            ids_all <-setdiff(ids_all, sample_ids_bar)    #ids_all[-sample_ids]
            cat("protected uid", head(d_wocat_chunk$UID),"\n")
            # All approaches
            new_d <- tryCatch(rbind(d_wocat_chunk,d_control_sample),error=function(e) return(NULL))
            # new_d <- tryCatch(rbind(d_wocat_chunk,d_control_all),error=function(e) return(NULL))
            #create a smaller distance matrix
            m_all <- tryCatch(match_wocat(new_d, pid=id_pa),error=function(e) return(NULL))
            # m_all <- match_wocat(new_d)
            m_all2 <- tryCatch(m_all[1,],error=function(e) return(NULL))
            # m_all2 <- m_all[1,]
            n_matches_temp <- tryCatch(nrow(m_all2$df),error=function(e) return(NULL))
            # n_matches_temp <- nrow(m_all2$df)
            if(!is.null(n_matches_temp)){
              # n_matches <- n_matches + nrow(m_all2$df)
              m_all2$df$pa_id <- rep(id_pa,n_matches_temp)
              m_all2_out <- rbind(m_all2_out, m_all2$df)
              matched_protected <- m_all2$df %>% dplyr::filter(status==TRUE)
              matched_control <- m_all2$df %>% dplyr::filter(status==FALSE)
              cat("matched_protected", nrow(matched_protected),"\n")
              n_matches <- n_matches + nrow(matched_protected)
              d_wocat_chunk <- d_wocat_chunk[-(match(matched_protected$UID,d_wocat_chunk$UID)),]
              # d_control_all <- d_control_all[-(match(matched_control$UID,d_control$UID)),]
            } 
            # ids_all <-setdiff(ids_all, sample_ids)
            ids_all0 <-setdiff(ids_all0, matched_control$UID)
            # else {
            #   n_treatment <- 0  #if not macthes are found in this sampling
            # }
          } else {n_treatment <- n_matches}
        }, error=function(e) return(NULL))
      # ids_all0 <-setdiff(ids_all0, matched_control$UID)
      match_score <- m_all2_out
      cat(table(match_score$status),"\n")
      pa_match <- rbind(pa_match,match_score)
    }
#  } else if (length(l)>=900){
  } else{
    pa_match <- NULL
  }

    output_filename <- paste(iso3,"_pa_", id_pa,"_matching_results_wk",gediwk,".RDS", sep="")
    saveRDS(pa_match, file=paste(f.path,"WDPA_matching_results/",iso3,"_wk",gediwk,"/",output_filename, sep=""))
  
  # Append the filename to the list
#  matched_PA <- c(matched_PA, output_filename)
#  print(output_filename)
                                       
  cat("Results exported for PA", id_pa, "\n")
  rm(pa_match)
  return(NULL)
}

tElapsed <- Sys.time()-startTime
# cat(tElapsed, "for matching all PAs in", iso3,"\n")
stopImplicitCluster()
cat("Done matching for",iso3,". Finishing...\n")

# Write the list of filenames to a CSV file
#output_filenames$matched_PA <- matched_PA
#write.csv(output_filenames, file=paste(f.path,"WDPA_matching_results/",iso3,"_wk",gediwk,"_matching_result_filenames.csv", sep=""), row.names = FALSE, col.names = FALSE)


In [19]:
f.path

[1] "/projects/my-public-bucket/GEDI_global_PA_v2/"

In [20]:
output_path <- paste("/projects/my-public-bucket/GEDI_global_PA_v2/WDPA_matching_results/",iso3,"_wk24",sep="")

In [21]:
list.files(output_path, full.names=F)

[1] "BaE_pa_101761_matching_results_wk24.RDS"   
  [2] "BaE_pa_10795_matching_results_wk24.RDS"    
  [3] "BaE_pa_10801_matching_results_wk24.RDS"    
  [4] "BaE_pa_10802_matching_results_wk24.RDS"    
  [5] "BaE_pa_10803_matching_results_wk24.RDS"    
  [6] "BaE_pa_115686_matching_results_wk24.RDS"   
  [7] "BaE_pa_115699_matching_results_wk24.RDS"   
  [8] "BaE_pa_115700_matching_results_wk24.RDS"   
  [9] "BaE_pa_115765_matching_results_wk24.RDS"   
 [10] "BaE_pa_115769_matching_results_wk24.RDS"   
 [11] "BaE_pa_115772_matching_results_wk24.RDS"   
 [12] "BaE_pa_19450_matching_results_wk24.RDS"    
 [13] "BaE_pa_19505_matching_results_wk24.RDS"    
 [14] "BaE_pa_19506_matching_results_wk24.RDS"    
 [15] "BaE_pa_19745_matching_results_wk24.RDS"    
 [16] "BaE_pa_19755_matching_results_wk24.RDS"    
 [17] "BaE_pa_198359_matching_results_wk24.RDS"   
 [18] "BaE_pa_198360_matching_results_wk24.RDS"   
 [19] "BaE_pa_198361_matching_results_wk24.RDS"   
 [20] "BaE_pa_198362_matching_results_wk24.RDS"   
 [21] "BaE_pa_198363_matching_results_wk24.RDS"   
 [22] "BaE_pa_2219_matching_results_wk24.RDS"     
 [23] "BaE_pa_31769_matching_results_wk24.RDS"    
 [24] "BaE_pa_31770_matching_results_wk24.RDS"    
 [25] "BaE_pa_31776_matching_results_wk24.RDS"    
 [26] "BaE_pa_33524_matching_results_wk24.RDS"    
 [27] "BaE_pa_33569_matching_results_wk24.RDS"    
 [28] "BaE_pa_33623_matching_results_wk24.RDS"    
 [29] "BaE_pa_33712_matching_results_wk24.RDS"    
 [30] "BaE_pa_33809_matching_results_wk24.RDS"    
 [31] "BaE_pa_33810_matching_results_wk24.RDS"    
 [32] "BaE_pa_33811_matching_results_wk24.RDS"    
 [33] "BaE_pa_33812_matching_results_wk24.RDS"    
 [34] "BaE_pa_33889_matching_results_wk24.RDS"    
 [35] "BaE_pa_33926_matching_results_wk24.RDS"    
 [36] "BaE_pa_33927_matching_results_wk24.RDS"    
 [37] "BaE_pa_33928_matching_results_wk24.RDS"    
 [38] "BaE_pa_33931_matching_results_wk24.RDS"    
 [39] "BaE_pa_33932_matching_results_wk24.RDS"    
 [40] "BaE_pa_33933_matching_results_wk24.RDS"    
 [41] "BaE_pa_33939_matching_results_wk24.RDS"    
 [42] "BaE_pa_33941_matching_results_wk24.RDS"    
 [43] "BaE_pa_33942_matching_results_wk24.RDS"    
 [44] "BaE_pa_33951_matching_results_wk24.RDS"    
 [45] "BaE_pa_33952_matching_results_wk24.RDS"    
 [46] "BaE_pa_33953_matching_results_wk24.RDS"    
 [47] "BaE_pa_33957_matching_results_wk24.RDS"    
 [48] "BaE_pa_33958_matching_results_wk24.RDS"    
 [49] "BaE_pa_33965_matching_results_wk24.RDS"    
 [50] "BaE_pa_33969_matching_results_wk24.RDS"    
 [51] "BaE_pa_33972_matching_results_wk24.RDS"    
 [52] "BaE_pa_33980_matching_results_wk24.RDS"    
 [53] "BaE_pa_33981_matching_results_wk24.RDS"    
 [54] "BaE_pa_33987_matching_results_wk24.RDS"    
 [55] "BaE_pa_33989_matching_results_wk24.RDS"    
 [56] "BaE_pa_33990_matching_results_wk24.RDS"    
 [57] "BaE_pa_33991_matching_results_wk24.RDS"    
 [58] "BaE_pa_33992_matching_results_wk24.RDS"    
 [59] "BaE_pa_33993_matching_results_wk24.RDS"    
 [60] "BaE_pa_33994_matching_results_wk24.RDS"    
 [61] "BaE_pa_33995_matching_results_wk24.RDS"    
 [62] "BaE_pa_33998_matching_results_wk24.RDS"    
 [63] "BaE_pa_34000_matching_results_wk24.RDS"    
 [64] "BaE_pa_34001_matching_results_wk24.RDS"    
 [65] "BaE_pa_34004_matching_results_wk24.RDS"    
 [66] "BaE_pa_34007_matching_results_wk24.RDS"    
 [67] "BaE_pa_34009_matching_results_wk24.RDS"    
 [68] "BaE_pa_34010_matching_results_wk24.RDS"    
 [69] "BaE_pa_34012_matching_results_wk24.RDS"    
 [70] "BaE_pa_34013_matching_results_wk24.RDS"    
 [71] "BaE_pa_34015_matching_results_wk24.RDS"    
 [72] "BaE_pa_34016_matching_results_wk24.RDS"    
 [73] "BaE_pa_34017_matching_results_wk24.RDS"    
 [74] "BaE_pa_34018_matching_results_wk24.RDS"    
 [75] "BaE_pa_34022_matching_results_wk24.RDS"    
 [76] "BaE_pa_34031_matching_results_wk24.RDS"    
 [77] "BaE_pa_34041_matching_results_wk24.RDS"    
 [78] "BaE_pa_34043_matching_results_wk24.RDS"    
 [79] "BaE_pa_34665_matc

In [22]:
write.csv(data.frame(list.files(output_path, full.names=F)),
          file=paste("/projects/my-public-bucket/GEDI_global_PA_v2/WDPA_matching_results/",iso3,"_wk24_filelist.csv",sep=""))

In [23]:
##
matching_results <- list.files(output_path, full.names=T)
length(matching_results)
##
for(i in 1:length(matching_results)){
    m_data <- readRDS(matching_results[i])
    print(nrow(m_data))
    }


[1] 266

[1] 17446
[1] 5422
[1] 15636
[1] 10584
[1] 6276
[1] 8498
[1] 0
[1] 186
[1] 428
[1] 450
[1] 2052
[1] 50
[1] 7744
[1] 6462
[1] 9114
[1] 474
[1] 11584
[1] 7752
[1] 2730
[1] 1892
[1] 7944
[1] 884
[1] 8808
[1] 3908
[1] 10562
[1] 420
[1] 0
[1] 382
[1] 1958
[1] 0
[1] 0
[1] 0
[1] 12074
[1] 17010
[1] 10566
[1] 5976
[1] 2336
[1] 3404
[1] 282
[1] 16
[1] 218
[1] 0
[1] 2194
[1] 12476
[1] 1604
[1] 3376
[1] 490
[1] 3102
[1] 1014
[1] 4
[1] 2
[1] 1282
[1] 2924
[1] 90
[1] 278
[1] 7092
[1] 5644
[1] 198
[1] 154
[1] 10002
[1] 15458
[1] 18730
[1] 29980
[1] 8688
[1] 65518
[1] 2206
[1] 1246
[1] 46688
[1] 78
[1] 7014
[1] 2286
[1] 388
[1] 20
[1] 438
[1] 66
[1] 162
[1] 1852
[1] 3134
[1] 5472
[1] 32880
[1] 14646
[1] 98356
[1] 23860
[1] 67208
[1] 8786
[1] 13446
[1] 0
[1] 4334
[1] 20
[1] 40
[1] 22
[1] 0
[1] 50456
[1] 14638
[1] 25030
[1] 6814
[1] 34
[1] 90
[1] 30
[1] 74
[1] 1856
[1] 1284
[1] 1120
[1] 1698
[1] 40704
[1] 25976
[1] 13694
[1] 4286
[1] 6616
[1] 17182
[1] 10684
[1] 28
[1] 7292
[1] 3904
[1] 258
[1] 468
[1]